# David Montero Loaiza

In [7]:
import keras
keras.__version__

'2.2.5'

# Using convnets with small datasets

The first thing to do is download the following file https://lara.web.cern.ch/lara/train.zip in the jupyter terminal and uncompress it in the same folder as this notebook. 

To download another dataset from imagenet you can do it with the URL list of the images and using `wget -i`



## Training a convnet from scratch

Training an image classification model with very little data is a common situation you will find yourself in if you end up doing Computer Vision in a professional context.  

Having "few" samples can mean anything from a few hundred to a few tens of thousands of images.  Let's illustrate a practical example here: let's focus on classifying images as "dogs" and "cats". 


## The importance of DEEP Learning in problems with few data

You may have heard many times that Deep Learning only works when you have large amounts of data. This is partly true: one of the characteristics of Deep learning is that you can find interesting features from the training dataset itself, and this a priori is easier when you have many examples available, especially in the case of having input datasets with a high dimensionality, such as images.

However, what constitutes a "large" dataset is relative. Specifically relative to the size and depth of the network we are trying to train. It is not possible to sand a convnet so that it becomes a complete problem with only a few dozen examples, but a few hundred can be enough if the model is well assembled (we will understand what "well assembled" means throughout the Deep Learning course).

Since convnets learn local characteristics, invariant under translations, they are very efficient in terms of the number of images needed to carry out perceptual problems. So training a convnet from 0 with a not very large dataset can still lead to reasonable results as we will see here.

But there is more: Deep Learning models are highly "recyclable". One can, for example, take an image classification problem and a trained speech-to-text converter on a very big dataset and then reuse it for solving a completely different problem only by adding some small modifications. More specifically, in the case of Computer Vision, many pre-trained models (usually trained on the ImageNet dataset) are made public so that one can download them and use them to create powerful Computer Vision models with very little data.

But here we will just run a simple example.
 


## Los datos

The cat vs dog dataset we use is not a Keras package. It was posted on Kaggle.com as part of a Computer Vision problem in late 2013, when ConvNets were not yet so popular. 

The images are medium resolution JPGEs. It looks like this:

![cats_vs_dogs_samples](https://s3.amazonaws.com/book.keras.io/img/ch5/cats_vs_dogs_samples.jpg)

It's no surprise that the 2013 Kaggle cat vs dog competition was won by ConvNets. The best were able to achieve up to 95% accuracy. In our example we are still far from this accuracy, but during the Deep Learning course we have learned how to approach this value using different methods to improve the performance of neural networks. It should be noted that in this example we are training on approximately only 10% of the data that was used for the contest. 
After downloading the dataset and decompressing it, we are going to create a new dataset containing three subsets: a training set containing 1000 images of each class, a validation set with 500 images of each class, and finally a test set with 500 images of each class.

Here we have a few lines of code that make us this distribution automatically:




In [0]:
from google.colab import drive

drive.mount('/content/drive')

!ls "/content/drive/My Drive/train"

In [0]:
import os, shutil

In [0]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = '/content/drive/My Drive/train'

# The directory where we will
# store our smaller dataset
base_dir = '/content/drive/My Drive/cats_and_dogs_small'
os.mkdir(base_dir)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# Directory with our validation cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# Directory with our validation dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

As a sanity check, let's count how many pictures we have in each training split (train/validation/test):

In [14]:
print('total training cat images:', len(os.listdir(train_cats_dir)))

total training cat images: 1000


In [15]:
print('total training dog images:', len(os.listdir(train_dogs_dir)))

total training dog images: 1000


In [16]:
print('total validation cat images:', len(os.listdir(validation_cats_dir)))

total validation cat images: 500


In [17]:
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

total validation dog images: 500


In [18]:
print('total test cat images:', len(os.listdir(test_cats_dir)))

total test cat images: 500


In [19]:
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total test dog images: 500


So effectively we have 2000 training images, 1000 validation images and 1000 test images. In each of these subsets there are the same number of examples from each class: this is what is called a balanced binary classification system, which means that our classification accuracy will be an adequate metric of the success of our solution.

## Building our network

In the above example we have built a small convnet to solve the problem of classifying handwritten numbers using the MNIST dataset, so we are already familiar with the terminology that keras uses. We are going to reuse the general structure we had in the previous example: our convnet will have a stack of alternate layers of `Conv2D` (with `relu` activation) and `MaxPooling2D` layers.

However, since we are dealing with larger images and a more complex problem, we will create our network accordingly: it will have one more layer of `Conv2D` + `MaxPooling2D`. This serves to increase the capacity of the network and to further reduce the size of the feature maps, so that they are not so huge when they reach the flattening step. We start using 150x150 input images (an arbitrary choice), and end up with feature maps that are 7x7 in size before the flattening layer.

It is important to note that the depth of feature maps progressively increases as we move through the neural network (from 32 to 128) while the size of feature maps decreases (from 148x148 to 7x7). You will see this pattern in almost all convnets.

As we are attacking a binary classification problem (dog or cat), we are going to finish the network with a single unit (a dense layer of size 1) and with a sigmoid activation. This unit will encode the probability that our network is looking at one class or another.

The final look of the model should be as follows:


![modelo_red_animales.png](https://github.com/laramaktub/MachineLearningI/blob/master/modelo_red_animales.png?raw=true)

In [9]:
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape = (150,150,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 15, 15, 128)      

Para el paso de compilación utilizaremos el optimizador `RMSprop`(lr=1e-4). Como nuestra red termina con una única unidad sigmoide, vamos a utilizar binary crossentropy como nuestra función de pérdida.

In [0]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr = 1e-4),loss="binary_crossentropy",metrics=['acc'])

##  Data preprocessing

The images must be properly formatted as float tensors before they are given to the net. That's just what we're going to do here. Before we pre-process them, the images are JPEG files. The steps to be able to give them to our network are roughly as follows:

* Read the files with the images.
* Decode the content of the JPEG in a "grid" with the RGB of the pixels 
* Turn that "grill" into floatation devices
* Rescale the pixel values (between 0 and 255) to the [0, 1] interval as neural networks prefer to work with small values. 

All this may seem very complicated but thanks to Keras our life is much easier and we can count on your tools to take care of these steps automatically. Keras has a module with tools for image processing, which can be found in `keras.preprocessing.image`. In particular, it contains the class `ImageDataGenerator` that allows us to automatically convert images we have on the hard disk into pre-processed tensors. This is exactly what we'll be using next. To do this we can use the flow_from_directory to take the images directly from the folders that we previously generated. We give it as input the folders where the training (or validation) images are, the size of the images (target_size), the size of the batch we're going to use (we're going to start with 20) and as there are only two categories, we tell it that we're going to use binary_crossentropy (class_mode). When we run these commands we'll get the following, the total number of images and how many classes there are.

In [21]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150, 150),batch_size=20,class_mode='binary')
validation_generator = validation_datagen.flow_from_directory(validation_dir,target_size=(150, 150),batch_size=20,class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir,target_size=(150, 150),batch_size=20,class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Let's take a look at one of these generators: it takes us to a batch of 150x150 RGB images (dimensions `(20, 150, 150, 3)`) and binary tags (dimension `(20,)`). 20 is the number of examples in each batch (what we call the batch size). The generator generates these batches indefinitely: it runs a loop endlessly through all the images we have in the folder. That's why we have to type `break` to break the loop at some point.


In [22]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


Now let's make the fit. In this case, as what we have is a generator, we use fit_generator. We are going to run 30 epochs and use the validation dataset.

In [23]:
history = model.fit_generator(generator=train_generator,epochs=30,validation_data=validation_generator)




Epoch 1/30





100/100 [==============================] - 111s 1s/step - loss: 0.6899 - acc: 0.5325 - val_loss: 0.6687 - val_acc: 0.6090
Epoch 2/30
100/100 [==============================] - 110s 1s/step - loss: 0.6526 - acc: 0.6195 - val_loss: 0.6601 - val_acc: 0.5780
Epoch 3/30
100/100 [==============================] - 110s 1s/step - loss: 0.6032 - acc: 0.6690 - val_loss: 0.7120 - val_acc: 0.5630
Epoch 4/30
100/100 [==============================] - 110s 1s/step - loss: 0.5647 - acc: 0.6975 - val_loss: 0.5897 - val_acc: 0.6950
Epoch 5/30
100/100 [==============================] - 110s 1s/step - loss: 0.5268 - acc: 0.7300 - val_loss: 0.5864 - val_acc: 0.6790
Epoch 6/30
100/100 [==============================] - 110s 1s/step - loss: 0.4975 - acc: 0.7570 - val_loss: 0.6251 - val_acc: 0.6610
Epoch 7/30
100/100 [==============================] - 110s 1s/step - loss: 0.4710 - acc: 0.7750 - val_loss: 0.5752 - val_acc: 0.7020
Epoch 8/30
100/100 [==============================] - 110s 1s

It's a nice idea to save the model after training

In [0]:
model.save('/content/drive/My Drive/model/cat_dog.h5')

Let's now evaluate our model using the test dataset

In [0]:
test_loss, test_acc = model.evaluate_generator(test_generator)

In [28]:
test_loss

1.078596914112568

In [29]:
test_acc

0.7329999995231629

Try to optimize the network with the tools you have learnt during the lesson. Try to make improvements both in terms of speed and accuracy. Comment the results. 

In [43]:
new_model = models.Sequential()

new_model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape = (150,150,3)))
new_model.add(layers.MaxPooling2D((2,2)))

new_model.add(layers.Conv2D(64,(3,3), activation='relu'))
new_model.add(layers.MaxPooling2D((2,2)))

new_model.add(layers.Conv2D(128,(3,3), activation='relu'))
new_model.add(layers.MaxPooling2D((2,2)))

new_model.add(layers.Conv2D(128,(3,3), activation='relu'))
new_model.add(layers.MaxPooling2D((2,2)))

new_model.add(layers.Flatten())

new_model.add(layers.Dropout(0.5))

new_model.add(layers.Dense(512,activation="relu"))
new_model.add(layers.Dense(1,activation="sigmoid"))

new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 15, 15, 128)      

In [0]:
new_model.compile(optimizer=optimizers.RMSprop(lr = 1e-4),loss="binary_crossentropy",metrics=['acc'])

In [45]:
from keras import callbacks

callback_es = callbacks.EarlyStopping(patience=8)

history = new_model.fit_generator(generator=train_generator,epochs=30,validation_data=validation_generator,callbacks = [callback_es])

Epoch 1/30
100/100 [==============================] - 109s 1s/step - loss: 0.6939 - acc: 0.5220 - val_loss: 0.6790 - val_acc: 0.5830
Epoch 2/30
100/100 [==============================] - 108s 1s/step - loss: 0.6725 - acc: 0.5825 - val_loss: 0.6527 - val_acc: 0.6280
Epoch 3/30
100/100 [==============================] - 108s 1s/step - loss: 0.6366 - acc: 0.6380 - val_loss: 0.6320 - val_acc: 0.6300
Epoch 4/30
100/100 [==============================] - 109s 1s/step - loss: 0.5968 - acc: 0.6830 - val_loss: 0.6472 - val_acc: 0.6370
Epoch 5/30
100/100 [==============================] - 110s 1s/step - loss: 0.5735 - acc: 0.7030 - val_loss: 0.6253 - val_acc: 0.6540
Epoch 6/30
100/100 [==============================] - 110s 1s/step - loss: 0.5587 - acc: 0.7105 - val_loss: 0.5775 - val_acc: 0.6970
Epoch 7/30
100/100 [==============================] - 109s 1s/step - loss: 0.5280 - acc: 0.7330 - val_loss: 0.5911 - val_acc: 0.6780
Epoch 8/30
100/100 [==============================] - 110s 1s/step - 

In [0]:
new_model.save('/content/drive/My Drive/model/cat_dog_callback_dropout.h5')

In [0]:
new_test_loss, new_test_acc = new_model.evaluate_generator(test_generator)

In [48]:
new_test_loss

0.6031138455867767

In [49]:
new_test_acc

0.7489999997615814